In [13]:
import sys
import os
import datetime
    
try: sc.stop()
except: pass

spark_home= '/opt/cloudera/parcels/CDH/lib/spark/'

os.environ ['SPARK_HOME'] = spark_home

sys.path.insert( 0, os.path.join( spark_home, 'python' ) )
sys.path.insert( 0, os.path.join( spark_home, 'python/lib/py4j-0.9-src.zip' ) )

from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.functions import *
from pyspark.sql.types import *

conf = SparkConf().setAppName( 'anton_jupyter' )\
                    .setMaster( 'yarn-client' )\
                    .setExecutorEnv('PATH', os.environ[ 'PATH' ] ) \
                    .set('spark.executor.core', '5' )\
                    .set('spark.executor.memory', '25g' )\
                    .set('spark.driver.core', '5' )\
                    .set('spark.driver.memory', '25g' )\
                    .set('spark.yarn.driver.memoryOverhead', '4096' ) \
                    .set('spark.yarn.executor.memoryOverhead', '4096' )\
                    .set('spark.kryoserializer.buffer.max', '2047')\
                    .set('spark.driver.maxResultSize', '8g')\
                    .set("spark.dynamicAllocation.enabled","true") \
                    .set("spark.dynamicAllocation.minExecutors","10") \
                    .set("spark.dynamicAllocation.maxExecutors","16") \
                    .set("spark.dynamicAllocation.initialExecutors","10") \
                    .set("spark.dynamicAllocation.executorIdleTimeout","60s") \
                    .set("spark.dynamicAllocation.schedulerBacklogTimeout","5s") \
                    .set("spark.dynamicAllocation.sustainedSchedulerBacklogTimeout","5s")

sc = SparkContext( conf=conf )
print("http://kratos29.ccaf.omega.sbrf.ru:8088/proxy/{}".format(sc.applicationId))
sqlContext = HiveContext( sc )
sqlContext.sql("use core_internal_rsa_pc");

http://kratos29.ccaf.omega.sbrf.ru:8088/proxy/application_1524839640503_13387


In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

plt.style.use('ggplot')

%matplotlib inline

In [15]:
from drop_finders import FingerprintFinder, HardwareFinder, PhoneFinder

In [16]:
from drop_finders import add_source_column, filter_by_user_count, get_flattened_dataframe

In [17]:
days = ['20180303']
day = days[0]
drops = ['4276400011584456', '2202200369649916']

In [18]:
path = '/home/dandreev/Andreev/Anton_tmp/neighborhood_scripts'

input_filename = 'drops.csv'

drops = pd.read_csv('{}/input/{}'.format(path, input_filename), index_col=None, header=None).values[:,0].tolist()
drops[:10]

['96062983',
 '51235053',
 '64496385',
 '91287542',
 'MBK5879145',
 '81110752',
 '97592231',
 '19084355',
 '11616784',
 '27984493']

In [19]:
finder = PhoneFinder(drops, 0.1, 30)

In [43]:
res = finder.get_drops_in_range(sqlContext, ['20180313'])

KeyError: 'phone'

In [46]:
df = finder.loaded
df

,phone,card_number


In [48]:
filter_by_user_count1(df, drops, 0.1, 30)

,phone,card_number


In [42]:
df.to_csv('empty')

In [27]:
df1 = df.copy()

In [41]:
df1 = df1.append({'phone': '89260010082','card_number': '123'}, ignore_index=True)
df1.empty

False

In [37]:
filter_by_user_count(df1, drops, 0.1, 30)

,phone,card_number


In [47]:
def filter_by_user_count1(df, drops, drop_users_part, max_users):
    if df.empty: return df
    
    index, source = df.columns
    grouped = df.groupby(index).agg({source: {
                                    'source_part' : lambda x: len(set(x) & set(drops))*1.0 / len(set(x)),
                                    'source_count': lambda x: len(set(x))}
                                    })[source].reset_index()
            
    index_filtered = grouped[index][(grouped['source_part'] > drop_users_part) & 
                                    (grouped['source_count'] <= max_users)]
    
    result = df[df[index].isin(index_filtered)].drop_duplicates()
    
    return result

In [197]:
res2 = filter_by_user_count1(df, drops, 0.1, 20)
res2 =res2.groupby('complex_index').agg({'user_id': {'all_users': len, \
                                                     'drops': lambda x: len((set(x) & set(drops)))}})\
                                                        ['user_id'].reset_index()
res2['part'] = res2.drops / res2.all_users
print(len(res2[res2.part != 1]))
res2[res2.part != 1].sort_values(by=['all_users'], ascending=True)

48


,complex_index,drops,all_users,part
73,yygkLRzVC4|37.113|20180505,1,2,0.500000
72,ywUUvv5FQw|178.62|20180505,1,2,0.500000
45,gCzyS40dxy|178.187|20180505,1,2,0.500000
47,hOD7sC9Qu8|213.110|20180505,1,2,0.500000
49,iRVwyUcpTr|5.164|20180505,1,2,0.500000
53,lMx8P9AC9y|95.85|20180505,1,2,0.500000
16,DT8qYxeC5w|62.118|20180505,1,2,0.500000
32,WZK6OaaTxB|176.109|20180505,1,2,0.500000
28,VtTM6y6ktd|94.41|20180505,1,2,0.500000
12,AndoUcTqz0|185.154|20180505,1,2,0.500000


In [168]:
res1 = filter_by_user_count(df, drops, 0.1)
res1 =res1.groupby('complex_index').agg({'user_id': {'all_users': len, 'drops': lambda x: len((set(x) & set(drops)))}})\
['user_id'].reset_index()
res1['part'] = res1.drops / res1.all_users
print(len(res1[res1.part != 1]))
res1[res1.part != 1].sort_values(by=['all_users'], ascending=True)

49


,complex_index,drops,all_users,part
74,yygkLRzVC4|37.113|20180505,1,2,0.500000
73,ywUUvv5FQw|178.62|20180505,1,2,0.500000
46,gCzyS40dxy|178.187|20180505,1,2,0.500000
48,hOD7sC9Qu8|213.110|20180505,1,2,0.500000
50,iRVwyUcpTr|5.164|20180505,1,2,0.500000
17,DT8qYxeC5w|62.118|20180505,1,2,0.500000
54,lMx8P9AC9y|95.85|20180505,1,2,0.500000
33,WZK6OaaTxB|176.109|20180505,1,2,0.500000
29,VtTM6y6ktd|94.41|20180505,1,2,0.500000
65,sm7FL0ul/C|87.239|20180505,1,2,0.500000
